In [11]:
import yfinance as yf
import pandas as pd
import ta
import os
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from openpyxl import load_workbook
from openpyxl.styles import PatternFill


import dash
from dash import dcc, html
import plotly.express as px
from us60stock_pool import us_stock_pool as stock_list

import importlib
import us10stock_pool
importlib.reload(us10stock_pool)



NameError: name 'null' is not defined

In [2]:
def download_all_data(stock_list, period="1y", interval="1d"):
    """
    一次性批量下载多个股票的数据，避免 yfinance 多次调用。
    返回字典，key=股票代码, value=DataFrame。
    """
    import yfinance as yf
    import pandas as pd

    if not stock_list:
        raise ValueError("股票列表不能为空")

    # 批量下载（group_by="ticker" 会分股票存储）
    try:
        data = yf.download(
            stock_list, period=period, interval=interval,
            auto_adjust=True, group_by="ticker", threads=True
        )
    except Exception as e:
        print(f"❌ 下载数据失败: {e}")
        return {}

    results = {}
    for ticker in stock_list:
        try:
            # 如果只有一个股票，yfinance 不会返回分组结构
            if isinstance(data.columns, pd.MultiIndex):
                df = data[ticker].dropna()
            else:
                df = data.dropna()

            if df.empty:
                print(f"⚠️ {ticker} 没有可用数据")
                continue

            results[ticker] = df
        except Exception as e:
            print(f"⚠️ {ticker} 数据处理失败: {e}")
            continue

    return results

# 使用方法
# stock_data = download_all_data(["AAPL", "MSFT", "TSLA"])
# print(stock_data["AAPL"].head())


In [3]:
# ---------------- 指标计算 ----------------
def calculate_indicators(data):
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)

    data = data.dropna(subset=["Close","Volume"], how="any")
    data = data[data["Volume"] > 0]

    data["Pct_Change"] = data["Close"].pct_change() * 100
    data["Vol_Avg20"] = data["Volume"].rolling(20).mean()
    data["Relative_Volume"] = data["Volume"] / data["Vol_Avg20"]

    data["RSI"] = ta.momentum.RSIIndicator(data["Close"], window=14).rsi()
    macd_indicator = ta.trend.MACD(data["Close"])
    data["MACD"] = macd_indicator.macd()
    data["MACD_Signal"] = macd_indicator.macd_signal()
    data["MA20"] = data["Close"].rolling(20).mean()
    data["MA50"] = data["Close"].rolling(50).mean()
    data["MA200"] = data["Close"].rolling(200).mean()
    bb = ta.volatility.BollingerBands(data["Close"])
    data["Upper_BB"] = bb.bollinger_hband()
    data["Lower_BB"] = bb.bollinger_lband()
    atr = ta.volatility.AverageTrueRange(data["High"], data["Low"], data["Close"])
    data["ATR"] = atr.average_true_range()

      # 前期数据
    data["Close_prev"] = data["Close"].shift(1)
    data["Open_prev"] = data["Open"].shift(1)
    data["High_prev"] = data["High"].shift(1)
    data["Low_prev"] = data["Low"].shift(1)
    data["High_prev2"] = data["High"].shift(2)
    data["Low_prev2"] = data["Low"].shift(2)
    data["High20"] = data["High"].rolling(20).max()
    data["Low20"] = data["Low"].rolling(20).min()

    return data


In [4]:
# ---------------- 策略库 (50个条件) ----------------
def check_strategies(m, price):
    signals = []

    # RSI
    if m["RSI"] < 30: signals.append("RSI超卖")
    if m["RSI"] > 70: signals.append("RSI超买")

    # MACD
    if m["MACD"] > m["MACD_Signal"]: signals.append("MACD金叉")
    if m["MACD"] < m["MACD_Signal"]: signals.append("MACD死叉")

    # 均线
    if price > m["MA20"]: signals.append("价格站上MA20")
    if price < m["MA20"]: signals.append("价格跌破MA20")
    if m["MA20"] > m["MA50"]: signals.append("MA20上穿MA50")
    if m["MA20"] < m["MA50"]: signals.append("MA20下穿MA50")
    if price > m["MA50"]: signals.append("站上MA50")
    if price < m["MA50"]: signals.append("跌破MA50")
    if m["MA20"] > m["MA50"] and m["MA50"] > m["MA200"]: signals.append("多头排列(MA20/50/200)")
    if m["MA20"] < m["MA50"] and m["MA50"] < m["MA200"]: signals.append("空头排列(MA20/50/200)")

    # 成交量
    if m["Volume"] > m["Vol_Avg20"]*2: signals.append("放量突破")
    if m["Volume"] < m["Vol_Avg20"]*0.5: signals.append("成交量萎缩")
    if m["Relative_Volume"] > 2: signals.append("相对成交量>2x")
    if m["Volume"] > m["Vol_Avg20"] and m["Pct_Change"] > 0: signals.append("价涨量增")
    if m["Volume"] < m["Vol_Avg20"] and m["Pct_Change"] < 0: signals.append("价跌量缩")

    # 涨跌幅
    if m["Pct_Change"] > 3: signals.append("突破涨幅>3%")
    if m["Pct_Change"] < -3: signals.append("单日大跌<-3%")

    # 布林带
    if price > m["Upper_BB"]: signals.append("布林带突破上轨")
    if price < m["Lower_BB"]: signals.append("布林带跌破下轨")
    if (m["Upper_BB"] - m["Lower_BB"]) < (0.05*price): signals.append("布林带收缩")
    if (m["Upper_BB"] - m["Lower_BB"]) > (0.1*price): signals.append("布林带扩张")

    # 波动率
    if m["ATR"] > 2: signals.append("高波动股票")
    if m["ATR"] < 1: signals.append("低波动股票")
    if abs(m["Pct_Change"]) > m["ATR"]: signals.append("ATR突破")

    # 趋势
    if m["MA20"] > m["MA50"] and price > m["MA20"]: signals.append("趋势向上")
    if m["MA20"] < m["MA50"] and price < m["MA20"]: signals.append("趋势向下")

    # 震荡
    if abs(m["Pct_Change"]) < 0.5: signals.append("震荡盘整")
    if abs(m["Pct_Change"]) > 2 and m["Volume"] > m["Vol_Avg20"]*1.5: signals.append("盘整突破")

    # 缺口
    if "Open" in m and "Close_prev" in m:
        if m["Open"] > m["Close_prev"]*1.02: signals.append("跳空高开")
        if m["Open"] < m["Close_prev"]*0.98: signals.append("跳空低开")

    # K线
    if "Open_prev" in m:
        if m["Close"] > m["Open"] and m["Open"] < m["Close_prev"] and m["Close"] > m["Open_prev"]:
            signals.append("看涨吞没")
        if m["Close"] < m["Open"] and m["Open"] > m["Close_prev"] and m["Close"] < m["Open_prev"]:
            signals.append("看跌吞没")

    if m["Close"] > m["Open"] and m["Low"] < m["Open"]*0.98: signals.append("锤子线")
    if m["Close"] < m["Open"] and m["High"] > m["Open"]*1.02: signals.append("射击之星")
    if abs(m["Close"]-m["Open"]) < (0.1*(m["High"]-m["Low"])): signals.append("十字星")

    if "High_prev" in m and "Low_prev" in m:
        if m["High"] < m["High_prev"] and m["Low"] > m["Low_prev"]: signals.append("内包日")
        if m["High"] > m["High_prev"] and m["Low"] < m["Low_prev"]: signals.append("外包日")

    if "High_prev2" in m and "Low_prev2" in m:
        if price < m["Close_prev"] and m["High_prev"] > m["High_prev2"]: signals.append("双顶")
        if price > m["Close_prev"] and m["Low_prev"] < m["Low_prev2"]: signals.append("双底")

    # 新高新低
    if "High20" in m and price >= m["High20"]: signals.append("20日新高")
    if "Low20" in m and price <= m["Low20"]: signals.append("20日新低")

    # 动量
    if m["Pct_Change"] > 1 and m["RSI"] > 60: signals.append("动能上行")
    if m["Pct_Change"] < -1 and m["RSI"] < 40: signals.append("动能下行")

    return signals


In [5]:



# ---------------- 资金流 / 相对强弱 / 分时 / 多时间框架 ----------------
def capital_flow(ticker):
    data = yf.download(ticker, period="5d", interval="1d", auto_adjust=True)
    if len(data) < 2: return 0
    return round((data["Close"].iloc[-1]-data["Close"].iloc[-2])*data["Volume"].iloc[-1]/1e6,2)

def relative_strength(ticker):
    spy = yf.download("SPY", period="6mo", interval="1d", auto_adjust=True)["Close"]
    stock = yf.download(ticker, period="6mo", interval="1d", auto_adjust=True)["Close"]
    if stock.empty or spy.empty: return 0
    return round((stock.pct_change().mean()-spy.pct_change().mean())*100,2)

def intraday_trend(ticker):
    data = yf.download(ticker, period="5d", interval="15m", auto_adjust=True)["Close"]
    if len(data) < 2: return 0
    return round((data.iloc[-1]/data.iloc[0]-1)*100,2)

def multi_timeframe_confirm(ticker):
    daily = yf.download(ticker, period="6mo", interval="1d", auto_adjust=True)["Close"]
    hourly = yf.download(ticker, period="1mo", interval="1h", auto_adjust=True)["Close"]
    m15 = yf.download(ticker, period="5d", interval="15m", auto_adjust=True)["Close"]

    if daily.empty or hourly.empty or m15.empty:
        return False

    daily_last = daily.iloc[-1].item()
    hourly_last = hourly.iloc[-1].item()
    m15_last = m15.iloc[-1].item()

    daily_mean = daily.mean().item()
    hourly_mean = hourly.mean().item()
    m15_mean = m15.mean().item()


    return (daily_last > daily_mean) and (hourly_last > hourly_mean) and (m15_last > m15_mean)


In [6]:
# ---------------- 回测模块 ----------------
# ---------------- 优化后的胜率回测模块 ----------------
def backtest_strategies_optimized(ticker, lookback=120, horizon=5):
    """
    胜率回测优化版：
    - 使用 rolling window（滑动窗口）计算策略触发后的表现
    - 不再只看下一天，而是看未来N天的平均收益
    - 提供 胜率 / 平均收益率 / 中位数收益率 / 样本数 四个指标
    - 排序逻辑：优先样本数，其次胜率，最后平均收益
    """
    import numpy as np
    import yfinance as yf
    import pandas as pd

    data = yf.download(ticker, period="1y", interval="1d", auto_adjust=True)
    if data is None or len(data) < lookback:
        return pd.DataFrame()

    data = calculate_indicators(data)
    results = {}

    for i in range(len(data) - horizon):
        today = data.iloc[i]
        future = data.iloc[i+1:i+1+horizon]["Close"]
        signals = check_strategies(today, today["Close"])

        if future.empty:
            continue

        future_return = (future.iloc[-1] / today["Close"] - 1) * 100  # 未来收益率（%）

        for sig in signals:
            if sig not in results:
                results[sig] = {"Signals": 0, "Wins": 0, "Returns": []}

            results[sig]["Signals"] += 1
            if future_return > 0:
                results[sig]["Wins"] += 1
            results[sig]["Returns"].append(future_return)

    stats = []
    for sig, val in results.items():
        if val["Signals"] > 0:
            win_rate = val["Wins"] / val["Signals"] * 100
            avg_return = np.mean(val["Returns"])
            median_return = np.median(val["Returns"])  # 🔹 新增：中位数收益率
            stats.append({
                "策略": sig,
                "胜率%": round(win_rate, 2),
                "平均收益%": round(avg_return, 2),
                "中位数收益%": round(median_return, 2),
                "样本数": val["Signals"]
            })

    return pd.DataFrame(stats).sort_values(
        by=["样本数", "胜率%", "平均收益%"],
        ascending=[False, False, False]
    )



In [7]:
# ---------------- 自动筛选模块 (严格版) ----------------
import yfinance as yf
import pandas as pd

def holly_ai_today(stock_list, lookback=120, horizon=5, top_n=3):
    results = []

    for ticker in stock_list:
        try:
            data = yf.download(ticker, period="1y", interval="1d", auto_adjust=True, progress=False)

            if data is None or len(data) < lookback:
                print(f"❌ 跳过：{ticker}（数据不足，仅 {len(data) if data is not None else 0} 行）")
                continue

            latest = data.iloc[-1]

            results.append({
                "股票": ticker,
                "价格": float(latest["Close"].iloc[0]) if hasattr(latest["Close"], "iloc") else float(latest["Close"]),
                "历史数据量": int(len(data)),
            })
            print(f"✅ 成功处理：{ticker}, 数据行数={len(data)}")

        except Exception as e:
            print(f"⚠️ 错误：{ticker}, {e}")

    df = pd.DataFrame(results)

    if not df.empty:
        # 排序时就不会报错了
        df = df.sort_values(by="价格", ascending=False).head(top_n)

    return df


In [8]:
# ---------------- 调用 ----------------
# ---------------- 自动学习模块优化版 ----------------
def backtest_strategies_optimized(ticker, lookback=120, horizon=5):
    """
    胜率回测优化版：
    - 使用 rolling window（滑动窗口）计算策略触发后的表现
    - 不再只看下一天，而是看未来N天的平均收益
    - 提供 胜率 / 平均收益率 / 中位数收益率 / 收益率标准差 / 最大回撤 / 样本数 六个指标
    - 排序逻辑：优先样本数，其次胜率，最后平均收益
    """
    import numpy as np
    import yfinance as yf
    import pandas as pd

    try:
        data = yf.download(ticker, period="1y", interval="1d", auto_adjust=True,
                           progress=False, threads=True)
    except Exception as e:
        print(f"⚠️ 下载 {ticker} 失败: {e}")
        return pd.DataFrame()

    if data is None or len(data) < lookback:
        return pd.DataFrame()

    data = calculate_indicators(data)
    results = {}

    for i in range(len(data) - horizon):
        today = data.iloc[i]
        future = data.iloc[i+1:i+1+horizon]["Close"]
        signals = check_strategies(today, today["Close"])

        if future.empty:
            continue

        returns = (future / today["Close"] - 1) * 100  # 未来收益率序列
        future_return = returns.iloc[-1]  # 最后一天收益率

        for sig in signals:
            if sig not in results:
                results[sig] = {"Signals": 0, "Wins": 0, "Returns": []}

            results[sig]["Signals"] += 1
            if future_return > 0:
                results[sig]["Wins"] += 1
            results[sig]["Returns"].extend(returns.tolist())

    stats = []
    for sig, val in results.items():
        if val["Signals"] > 0:
            win_rate = val["Wins"] / val["Signals"] * 100
            avg_return = np.mean(val["Returns"])
            median_return = np.median(val["Returns"])
            std_return = np.std(val["Returns"])  # 收益率标准差
            max_drawdown = np.min(val["Returns"])  # 最大回撤（最差收益率）
            stats.append({
                "策略": sig,
                "胜率%": round(win_rate, 2),
                "平均收益%": round(avg_return, 2),
                "中位数收益%": round(median_return, 2),
                "标准差%": round(std_return, 2),
                "最大回撤%": round(max_drawdown, 2),
                "样本数": val["Signals"]
            })

    return pd.DataFrame(stats).sort_values(
        by=["样本数", "胜率%", "平均收益%"],
        ascending=[False, False, False]
    )


In [10]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
from us30stock_pool import us_stock_pool
from holly_ai_today import holly_ai_today   # 你的函数


# ✅ 直接获取真实数据
df = holly_ai_today(us_stock_pool, top_n=10)
print("传给Dash的股票池：", df)

# 绘制图表
fig_bar = px.bar(
    df, x="股票", y="AI预测上涨概率%", color="历史胜率%",
    hover_data=["价格", "信号数量", "历史胜率%", "盈亏比(R/R)", "建议"],
    title="📊 Holly AI 自动学习推荐"
)

fig_scatter = px.scatter(
    df, x="历史胜率%", y="AI预测上涨概率%",
    size="盈亏比(R/R)", color="股票", hover_name="触发信号",
    title="信号分布图"
)

# ---------------- Dash 应用 ----------------
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1("📈 Holly AI 股票推荐", style={"textAlign": "center", "marginBottom": "20px"}),

    # 综合推荐前三名
    html.Div([
        html.H3("🔥 综合推荐前三名"),
        html.Table([
            html.Thead(html.Tr([html.Th(col) for col in df.head(3).columns], style={"backgroundColor": "#007BFF", "color": "white"})),
            html.Tbody([
                html.Tr([
                    html.Td(df.head(3).iloc[i][col]) for col in df.head(3).columns
                ], style={"backgroundColor": "#f9f9f9" if i%2==0 else "#ffffff", "textAlign": "center"}) for i in range(len(df.head(3)))
            ])
        ])
    ]),

    # 柱状图
    dcc.Graph(figure=fig_bar),

    # 散点图
    dcc.Graph(figure=fig_scatter),

    # 推荐详情表
    html.Div([
        html.H3("推荐详情表"),
        html.Table([
            html.Thead(html.Tr([html.Th(col) for col in df.columns], style={"backgroundColor": "#343a40", "color": "white"})),
            html.Tbody([
                html.Tr([
                    html.Td(df.iloc[i][col]) for col in df.columns
                ], style={"backgroundColor": "#f9f9f9" if i%2==0 else "#ffffff", "textAlign": "center"}) for i in range(len(df))
            ])
        ])
    ])
])

if __name__ == "__main__":
    app.run(debug=True, port=8051)


✅ 成功处理：AMZN, 最新价=232.3300018310547, 数据行数=250
✅ 成功处理：NVDA, 最新价=167.02000427246094, 数据行数=250
✅ 成功处理：META, 最新价=752.4500122070312, 数据行数=250
✅ 成功处理：TSLA, 最新价=350.8399963378906, 数据行数=250
✅ 成功处理：AVGO, 最新价=334.8900146484375, 数据行数=250
✅ 成功处理：JPM, 最新价=294.3800048828125, 数据行数=250
✅ 成功处理：BAC, 最新价=49.77000045776367, 数据行数=250
✅ 成功处理：GS, 最新价=738.2100219726562, 数据行数=250
✅ 成功处理：MS, 最新价=148.08999633789062, 数据行数=250
✅ 成功处理：V, 最新价=343.2200012207031, 数据行数=250
✅ 成功处理：MA, 最新价=584.219970703125, 数据行数=250
✅ 成功处理：WMT, 最新价=100.51000213623047, 数据行数=250
✅ 成功处理：COST, 最新价=963.47998046875, 数据行数=250
✅ 成功处理：PG, 最新价=160.02000427246094, 数据行数=250
✅ 成功处理：KO, 最新价=67.95999908447266, 数据行数=250
✅ 成功处理：PEP, 最新价=146.38999938964844, 数据行数=250
✅ 成功处理：NKE, 最新价=73.91000366210938, 数据行数=250
✅ 成功处理：MCD, 最新价=314.3800048828125, 数据行数=250
✅ 成功处理：SBUX, 最新价=85.43000030517578, 数据行数=250
✅ 成功处理：JNJ, 最新价=178.42999267578125, 数据行数=250
✅ 成功处理：PFE, 最新价=24.8799991607666, 数据行数=250
✅ 成功处理：UNH, 最新价=315.3900146484375, 数据行数=250
✅ 成功处理：ABBV, 最新价=212.559997558